### Installation
Install the packages required for executing this notebook.

This is a variation of Lab 4.1 using Artifacts Instead of Dicts as we did in Lab 5 Ex1 and Ex2. In addition, only the best model is uploaded.   

In [3]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q
! pip3 install -U google-cloud-storage {USER_FLAG} -q
! pip3 install {USER_FLAG} kfp google-cloud-pipeline-components --upgrade -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 1.8.14 requires google-cloud-storage<2,>=1.20.0, but you have google-cloud-storage 2.5.0 which is incompatible.
google-cloud-pipeline-components 1.0.25 requires google-cloud-storage<2,>=1.20.0, but you have google-cloud-storage 2.5.0 which is incompatible.


## Restart the kernel
Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [4]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

Check the versions of the packages you installed. The KFP SDK version should be >=1.6.

In [1]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"

KFP SDK version: 1.8.14


In [1]:
import kfp
import typing
from typing import Dict
from typing import NamedTuple
from kfp.v2 import dsl
from kfp.v2.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)
import google.cloud.aiplatform as aip
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2.components import importer_node
from google_cloud_pipeline_components.types import artifact_types

#### Project and Pipeline Configurations

In [39]:
#The Google Cloud project that this pipeline runs in.
project_id = "your project id"
# The region that this pipeline runs in
region = "us-central1"
# Specify a Cloud Storage URI that your pipelines service account can access. The artifacts of your pipeline runs are stored within the pipeline root.
pipeline_root_path = "your pipeline root, gs://de_jads_temp"

#### Create Pipeline Components

We can create a component from Python functions (inline) and from a container. We will first try inline python functions. 

Step 1: Define the python function

Step 2:  Use **kfp.components.create_component_from_func** build the component. This function takes four parameters.

**1.func**: The Python function to convert.

**2.base_image**: (Optional.) Specify the Docker container image to run this function in. 

**3.output_component_file**: (Optional.) Writes your component definition to a file. 

**4.packages_to_install**: (Optional.) A list of versioned Python packages to install before running your function.

Another thing we need to consider is passing parameters between components. We can pass simple parameters such as integer, string, tuple, dict, and list by values. To pass the large datasets or complex configurations, we can use files. We can annotate the Python function’s parameters to indicate input or output files for the component. 

Refer to  https://www.kubeflow.org/docs/components/pipelines/sdk/python-function-components/ for more information.

#### Pipeline Component : Data Ingestion

In [3]:
@component(
    packages_to_install=["pandas","google-cloud-storage"],
    base_image="python:3.10.7-slim",
    output_component_file="download_data_3.yaml"
)
def download_data(project_id: str, bucket: str, file_name: str, dataset: Output[Dataset]):
    '''download data'''
    from google.cloud import storage
    import pandas as pd
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    # Downloaing the file from a google bucket 
    client = storage.Client(project=project_id)
    bucket = client.get_bucket(bucket)
    blob = bucket.blob(file_name)
    blob.download_to_filename(dataset.path + ".csv")
    logging.info('Downloaded Data!')

#### Pipeline Component : Training-MLP 

In [4]:
@component(
    packages_to_install=['pandas', 'keras', 'tensorflow', 'h5py'],
    base_image="python:3.10.7-slim",
    output_component_file="training_mlp_3.yaml"
)
def train_mlp (features: Input[Dataset], out_model: Output[Model]) -> dict:
    '''train a MLP with default parameters'''
    import pandas as pd
    from google.cloud import storage
    from keras.layers import Dense
    from keras.models import Sequential
    import json
    import logging 
    import sys
    import os
        
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df = pd.read_csv(features.path+".csv")
    
    logging.info(df.columns)
        
    # split into input (X) and output (Y) variables
    X = df.loc[:, ['ntp','age', 'bmi', 'dbp', 'dpf', 'pgc', 'si', 'tsft']].values
    Y = df.loc[:, ['class']].values
    # define model
    model_mlp = Sequential()
    model_mlp.add(Dense(12, input_dim=8, activation='relu'))
    model_mlp.add(Dense(8, activation='relu'))
    model_mlp.add(Dense(1, activation='sigmoid'))
    # compile model
    model_mlp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model_mlp.fit(X, Y, epochs=150, batch_size=10, verbose=0)
    # evaluate the model
    scores = model_mlp.evaluate(X, Y, verbose=0)
    logging.info(model_mlp.metrics_names)
    metrics = {
        "accuracy": scores[1],
        "loss": scores[0],
    }   
    logging.info(metrics)   
    
    out_model.metadata["file_type"] = ".h5"
    out_model.metadata["algo"] = "mlp"
    # Save the model
    m_file = out_model.path + ".h5"
    model_mlp.save(m_file)
    return metrics

#### Pipeline Component : Training LogisticRegression

In [5]:
@component(
    packages_to_install=['pandas', 'scikit-learn'],
    base_image="python:3.10.7-slim",
    output_component_file="train_lr_model_3.yaml"
)
def train_lr (features: Input[Dataset], out_model: Output[Model]) -> dict:
    '''train a LogisticRegression with default parameters'''
    import pandas as pd
    from sklearn.linear_model import LogisticRegression
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    from google.cloud import storage
    import json
    import logging 
    import sys
    import os
    import pickle  
       
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df = pd.read_csv(features.path+".csv")
    
    logging.info(df.columns)        
    
    x_train, x_test, y_train, y_test = train_test_split(df.drop('class',axis=1), 
                                                    df['class'], test_size=0.30, 
                                                    random_state=101)
    model_lr = LogisticRegression()
    model_lr.fit(x_train,y_train)

    metrics = {
        "accuracy": model_lr.score(x_test, y_test)
    }
    logging.info(metrics)  
    
    out_model.metadata["file_type"] = ".pkl"
    out_model.metadata["algo"] = "lr"
   # Save the model
    m_file = out_model.path + ".pkl"
    with open(m_file, 'wb') as f:  
        pickle.dump(model_lr, f)   
    return metrics

#### Pipeline Component : Prediction-MLP

In [6]:
@component(
    packages_to_install=['pandas', 'keras', 'tensorflow', 'h5py'],
    base_image="python:3.10.7-slim",
    output_component_file="predict_mlp_3.yaml"
)
def predict_mlp(model: Input[Model], features: Input[Dataset], results: Output[Dataset]):
    import pandas as pd
    from keras.models import load_model
    from google.cloud import storage
    import json
    import logging
    import sys
    import os
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df = pd.read_csv(features.path+".csv")
    
    xNew = df.loc[:, ['ntp', 'age', 'bmi', 'dbp', 'dpf', 'pgc', 'si', 'tsft']].values
    #load the model
    model_mlp = load_model(model.path + '.h5')
    dfcp = df.copy()
    result = model_mlp.predict(xNew)   
    y_classes = result.argmax(axis=-1)
    logging.info(y_classes)
    dfcp['pclass'] = y_classes.tolist()
    dfcp.to_csv(results.path + ".csv" , index=False, encoding='utf-8-sig')

#### Pipeline Component : Prediction-LR

In [17]:
@component(
    packages_to_install=['pandas', 'scikit-learn'],
    base_image="python:3.10.7-slim",
    output_component_file="predict_lr.yaml"
)
def predict_lr(model: Input[Model], features: Input[Dataset], results: Output[Dataset]):
    import pandas as pd
    import pickle  
    from google.cloud import storage
    import json
    import logging
    import sys
    import os
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df = pd.read_csv(features.path+".csv")
       
    filename = model.path + ".pkl"
        
    #Loading the saved model
    model_lr = pickle.load(open(filename, 'rb'))

    xNew = df[['ntp', 'age', 'bmi', 'dbp', 'dpf', 'pgc', 'si', 'tsft']]

    dfcp = df.copy()   
    y_classes = model_lr.predict(xNew)
    logging.info(y_classes)
    dfcp['pclass'] = y_classes.tolist()     
    dfcp.to_csv(results.path + ".csv" , index=False, encoding='utf-8-sig')

#### Pipeline Component : Algorithm Selection 

In [36]:
@component(
    base_image="python:3.10.7-slim",
    output_component_file="compare_model_v3.yaml"
)
def compare_model(mlp_metrics: dict, lr_metrics: dict) -> str:
    import logging
    import json
    import sys
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    logging.info(mlp_metrics)
    logging.info(lr_metrics)
    if mlp_metrics.get("accuracy") > lr_metrics.get("accuracy"):
        return "MLP"
    else :
        return "LR"

### Upload Model and Metrics to Google Bucket 

In [28]:
@component(
    packages_to_install=["google-cloud-storage"],
    base_image="python:3.10.7-slim",
    output_component_file="model_upload_gsc_3.yaml"
)
def upload_model_to_gcs(project_id: str, model_repo: str, model: Input[Model]):
    '''upload model to gsc'''
    from google.cloud import storage   
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)    
  
    # upload the model to GCS
    client = storage.Client(project=project_id)
    bucket = client.get_bucket(model_repo)
    blob = bucket.blob(str(model.metadata["algo"]) + '_model' + str(model.metadata["file_type"])) 
    blob.upload_from_filename(model.path + str(model.metadata["file_type"]))       
    
    print("Saved the model to GCP bucket : " + model_repo)

#### Define the Pipeline

In [33]:
# Define the workflow of the pipeline.
@kfp.dsl.pipeline(
    name="diabetes-prdictor-training-pipeline-v3",
    pipeline_root=pipeline_root_path)
def pipeline(project_id: str, data_bucket: str, trainset_filename: str, model_repo: str, testset_filename: str):
    
    
    di_op = download_data(
        project_id=project_id,
        bucket=data_bucket,
        file_name=trainset_filename
    )

 
    training_mlp_job_run_op = train_mlp(
        features=di_op.outputs["dataset"]
    )
    
     
    training_lr_job_run_op = train_lr(
        features=di_op.outputs["dataset"]
    )
    
    pre_di_op = download_data(
        project_id=project_id,
        bucket=data_bucket,
        file_name=testset_filename
    ).after(training_mlp_job_run_op, training_lr_job_run_op)
        
        
    comp_model__op = compare_model(training_mlp_job_run_op.outputs["output"],
                                       training_lr_job_run_op.outputs["output"]).after(training_mlp_job_run_op, training_lr_job_run_op)  
    
    # defining the branching condition
    with dsl.Condition(comp_model__op.output=="MLP"):
        predict_mlp_job_run_op = predict_mlp(
            model=training_mlp_job_run_op.outputs["out_model"],      
            features=pre_di_op.outputs["dataset"]
        )
        upload_model_mlp_to_gc_op = upload_model_to_gcs(
            project_id=project_id,
            model_repo=model_repo,
            model=training_mlp_job_run_op.outputs['out_model']
        ).after(predict_mlp_job_run_op)
        
    with dsl.Condition(comp_model__op.output=="LR"):
        predict_lr_job_run_op = predict_lr(
            model=training_lr_job_run_op.outputs["out_model"],     
            features=pre_di_op.outputs["dataset"]
        )
        upload_model_lr_to_gc_op = upload_model_to_gcs(
            project_id=project_id,
            model_repo=model_repo,
            model=training_lr_job_run_op.outputs['out_model']
        ).after(predict_lr_job_run_op)  

#### Compile the pipeline into a JSON file

In [37]:
from kfp.v2 import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='diabetes_prdictor_training_pipeline_v3.json')

#### Submit the pipeline run

In [38]:
import google.cloud.aiplatform as aip

job = aip.PipelineJob(
    display_name="diabetes-predictor",
    enable_caching=False,
    template_path="diabetes_prdictor_training_pipeline_v3.json",
    pipeline_root=pipeline_root_path,
    parameter_values={
        'project_id': project_id, # makesure to use your project id 
        'data_bucket': 'data_de2022',  # makesure to use your data bucket name 
        'trainset_filename': 'training_set.csv',     # makesure to upload these to your data bucket from DE2022/lab4/data
        'testset_filename': 'prediction_set.csv',    # makesure to upload these to your data bucket from DE2022/lab4/data
        'model_repo':'model_repo_de2022' # makesure to use your model bucket name 
    }
)

job.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/958343845263/locations/us-central1/pipelineJobs/diabetes-prdictor-training-pipeline-v3-20221021142530
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/958343845263/locations/us-central1/pipelineJobs/diabetes-prdictor-training-pipeline-v3-20221021142530')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/diabetes-prdictor-training-pipeline-v3-20221021142530?project=958343845263
PipelineJob projects/958343845263/locations/us-central1/pipelineJobs/diabetes-prdictor-training-pipeline-v3-20221021142530 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/958343845263/locations/us-central1/pipelineJobs/diabetes-prdictor-training-pipeline-v3-20221021142530 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/958343845263/locations/us-central1/pipelineJobs/diabetes-prdictor-training-pipeline-v3-